In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import json
from pydantic import BaseModel, Field
import string
import re
from typing import Optional 

/Users/catherineyeh/.local/share/virtualenvs/story-viz-mNgXibik/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
try:
    with open("../secrets.json") as f:
        secrets = json.load(f)
    open_ai_key = secrets["openai"]
    anthropic_key = secrets["anthropic"]
    google_key = secrets["google"]
    os.environ["OPENAI_API_KEY"] = open_ai_key
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key
    os.environ["GOOGLE_API_KEY"] = google_key
    print("API key loaded.")
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [5]:
def format_gemini_response(res):
    output_formatted = res.content[7:-3]
    # turn into json
    output = json.loads(output_formatted)
    return output

class DotDict:
    # allow dot notation for dicts
    def __init__(self, dictionary):
        # check if dictionary is a string
        if isinstance(dictionary, str):
            # if it is a string, convert it to a dictionary
            print(dictionary)
            dictionary = json.loads(dictionary)
        for key, value in dictionary.items():
            setattr(self, key, value)

In [6]:
# model_type = "default"
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
llm2 = ChatAnthropic(model_name="claude-3-5-sonnet-20240620", temperature=0.1, api_key=os.getenv("ANTHROPIC_API_KEY"))

model_type = "gemini"
llm3 = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.1)

# safety_settings={
#     HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
# }

# # original method
# og_generate = ChatGoogleGenerativeAI._generate

# # patch

# ChatGoogleGenerativeAI._generate = partial(llm._generate, safety_settings=safety_settings)

In [7]:
# Pydantic
class StoryInfo(BaseModel):
    """Get overall information about the story."""
    firstline: str = Field(description="First line of the story")
    lastline: Optional[str] = Field(description="Last line of the story or first volume")
    markers: list[str] = Field(description="List of markers to split the story into chapters. If there are multiple volumes, focus on the first volume.")

def get_marker_prompt(story):
    prompt = f'''List the markers needed to split this story into chapters: {story}. Format your response like Story Info:
    class StoryInfo(BaseModel):
        """Get overall information about the story."""
        firstline: str = Field(description="First line of the story")
        lastline: Optional[str] = Field(description="Last line of the story or first volume")
        markers: list[str] = Field(description="List of markers to split the story into chapters. Keep all punctuation marks and capitalization.")
    '''
    return prompt

# structured_llm = llm2.with_structured_output(StoryInfo)
structured_llm = llm.with_structured_output(StoryInfo)

## split text into chapters

In [8]:
story_info = {
    "title": "Frankenstein",
    "type": "Book",
    "author": "Mary Shelley",
    "year": 1818,
    "url": "https://www.gutenberg.org/ebooks/84",
    "image": "https://images.penguinrandomhouse.com/cover/9780143131847",
}

In [9]:
# read in txt file from scripts folder
story_name = "frankenstein"
with open(f"scripts/{story_name}.txt") as f:
    story = f.read()

In [8]:
# focus on the first volume if there are multiple volumes
if "END OF VOL." in story:
    volumes = story.split("END OF VOL.")
    story = volumes[0]

In [9]:
# split story into chunks
char_chunk = 120000
print(len(story))
story_chunks = [story[i:i + char_chunk] for i in range(0, len(story), char_chunk)]
print(len(story_chunks))

438806
4


In [10]:
# use full story
story_chunks = [story]

In [33]:
all_res = []
for i, chunk in enumerate(story_chunks):
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            # if model_type == 'gemini':
            #     prompt = get_marker_prompt(story)
            #     markers = llm3.invoke(prompt)
            #     res = format_gemini_response(markers)
            #     print(res)
            #     res = DotDict(res['Story Info'])
            # else:
            res = structured_llm.invoke(chunk)
            all_res.append(res)
            print(f"Chunk {i}")
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True
    # if i < len(story_chunks) - 1:
    #     # Pause for a specified time (e.g., 60 seconds) to avoid rate limits
    #     time.sleep(60)  # Adjust the sleep duration as per your rate limit requirements

Chunk 0


In [34]:
# save first line from first chunk, last line from last chunk, and combine markers
first_line = all_res[0].firstline
last_line = all_res[-1].lastline
markers = []
for res in all_res:
    markers.extend(res.markers)

markers

['Letter 1',
 'Letter 2',
 'Letter 3',
 'Letter 4',
 'Chapter 1',
 'Chapter 2',
 'Chapter 3',
 'Chapter 4',
 'Chapter 5',
 'Chapter 6',
 'Chapter 7',
 'Chapter 8',
 'Chapter 9',
 'Chapter 10',
 'Chapter 11',
 'Chapter 12',
 'Chapter 13',
 'Chapter 14',
 'Chapter 15',
 'Chapter 16',
 'Chapter 17',
 'Chapter 18',
 'Chapter 19',
 'Chapter 20',
 'Chapter 21',
 'Chapter 22',
 'Chapter 23',
 'Chapter 24']

In [114]:
# first_line = res.firstline
# last_line = res.lastline
# markers = res.markers

In [35]:
# remove any marker that doesn't include one of these words
exclude_words = ["part", "preface", "contents", "by", "end", "epilogue", "letter"]
markers = [m for m in markers if not any(word in m.lower() for word in exclude_words)]

In [36]:
# save as json 
story_json = {
    "first_line": first_line,
    "last_line": last_line,
    "markers": markers
}
if not os.path.exists(f"json/{story_name}"):
    os.makedirs(f"json/{story_name}")
with open(f"json/{story_name}/summary.json", "w") as f:
    json.dump(story_json, f, indent=4)

In [10]:
story = re.sub(r' {2,}', ' ', story)  # Replace consecutive spaces with one space
story = re.sub(r'\n{2,}', '\n', story)  # Replace consecutive newlines with one newline
story = re.sub(r' ?\n ?', '\n', story)  # Replace space + newline or newline + space with one newline
story = story.replace("“", "\"").replace("”", "\"").replace("‘", "'").replace("’", "'")

In [11]:
story_formatted = story.replace("\n", " ")
# replace weird quotes too
story_formatted = story_formatted.replace("\"", " ").replace("'", " ")

In [12]:
# read in markers from summary.json
with open(f"json/{story_name}/summary.json") as f:
    summary = json.load(f)
    markers = summary["markers"]
    first_line = summary["first_line"]
    last_line = summary["last_line"]

In [12]:
story_formatted[-100:] 

' help produce our new eBooks, and how to subscribe to our email newsletter to hear about new eBooks.'

In [41]:
story_formatted.find(last_line)
last_line

'This website includes information about Project Gutenberg™, including how to make donations to the Project Gutenberg Literary Archive Foundation, how to help produce our new eBooks, and how to subscribe to our email newsletter to hear about new eBooks.'

In [13]:
# split story into chapters based on markers
chapters = []
cur_first_line = first_line.replace("\"", " ").replace("'", " ")
cur_min_index = 0
for i, marker in enumerate(markers):
    exists_next = i+1 < len(markers)
    next_marker = markers[i+1] if exists_next else ""
    
    if exists_next and " ACT " in next_marker.upper():
        next_marker = next_marker.upper()
    formatted_last_line = next_marker if exists_next else last_line

    if " ACT " not in next_marker.upper():
        formatted_last_line += " "
    cur_last_line = formatted_last_line
    cur_last_line = cur_last_line.replace("\"", " ").replace("'", " ")
    first_line_index = story_formatted.find(cur_first_line, cur_min_index)

    if i != 0 and " ACT " not in marker.upper():
        first_line_index += len(cur_first_line)
    last_line_index = story_formatted.find(cur_last_line, first_line_index)
    if i+1 == len(markers):
        if last_line_index == -1:
            last_line_index = len(story_formatted)
        else:
            last_line_index += len(cur_last_line) 
    chapter = story[first_line_index:last_line_index].strip()

    # add line number in front of each line
    chapter = chapter.split("\n")
    for j, line in enumerate(chapter):
        chapter[j] = f"LINE {j+1}: {line}"
    chapter = "\n".join(chapter)
    chapters.append(chapter)
    cur_first_line = cur_last_line
    cur_min_index = last_line_index

In [19]:
# check results
for i, marker in enumerate(markers):
    print(f"Chapter {marker}")
    chapter_first_line = chapters[i][:100]
    chapter_last_line = chapters[i][-100:]
    print("first_line:", chapter_first_line)
    print("last_line:", chapter_last_line)
    print("\n")

Chapter Chapter 1
first_line: LINE 1: You will rejoice to hear that no disaster has accompanied the
LINE 2: commencement of an ent
last_line: on in which she stood to me—my more than
LINE 628: sister, since till death she was to be mine only.


Chapter Chapter 2
first_line: LINE 1: We were brought up together; there was not quite a year difference in
LINE 2: our ages. I ne
last_line: tiny was too potent, and her immutable laws had decreed my utter and
LINE 189: terrible destruction.


Chapter Chapter 3
first_line: LINE 1: When I had attained the age of seventeen my parents resolved that I
LINE 2: should become a 
last_line: sted, and I took my leave.
LINE 228: Thus ended a day memorable to me; it decided my future destiny.


Chapter Chapter 4
first_line: LINE 1: From this day natural philosophy, and particularly chemistry, in the
LINE 2: most comprehens
last_line: ncipient disease; and I promised myself
LINE 216: both of these when my creation should be complete.


Chapter Chapter 5
fi

In [44]:
# save to new txt files
# create folder for story if it doesn't exist
if not os.path.exists(f"chapters/{story_name}"):
    os.makedirs(f"chapters/{story_name}")
for i, marker in enumerate(markers):
    with open(f"chapters/{story_name}/{marker}.txt", "w") as f:
        f.write(chapters[i])
        print(f"Chapter {marker} saved.")

Chapter Chapter 1 saved.
Chapter Chapter 2 saved.
Chapter Chapter 3 saved.
Chapter Chapter 4 saved.
Chapter Chapter 5 saved.
Chapter Chapter 6 saved.
Chapter Chapter 7 saved.
Chapter Chapter 8 saved.
Chapter Chapter 9 saved.
Chapter Chapter 10 saved.
Chapter Chapter 11 saved.
Chapter Chapter 12 saved.
Chapter Chapter 13 saved.
Chapter Chapter 14 saved.
Chapter Chapter 15 saved.
Chapter Chapter 16 saved.
Chapter Chapter 17 saved.
Chapter Chapter 18 saved.
Chapter Chapter 19 saved.
Chapter Chapter 20 saved.
Chapter Chapter 21 saved.
Chapter Chapter 22 saved.
Chapter Chapter 23 saved.
Chapter Chapter 24 saved.


## analyze scene

In [12]:
# Pydantic
class SceneInfo(BaseModel):
    """Get information about a scene in the story."""
    scene: str = Field(description="Short title of scene (don't include chapter name or number). Each scene should have a unique title, and each title should be no longer than 6 words.")
    summary: str = Field(description="1 line summary of the scene")
    firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
    lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

class SceneListInfo(BaseModel):
    """Get overall information about the story."""
    scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")

def get_scenelist_prompt(chapter):
    scene_prompt = f'''List the key scenes in this chapter: {chapter}. Format your response like SceneListInfo:
                    class SceneInfo(BaseModel):
                        """Get information about a scene in the story."""
                        scene: str = Field(description="Title of scene (don't include chapter name or number)")
                        summary: str = Field(description="1 line summary of the scene")
                        firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
                        lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

                    class SceneListInfo(BaseModel):
                        """Get overall information about the story."""
                        scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")'''
    return scene_prompt

scene_llm = llm.with_structured_output(SceneListInfo)

In [13]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in this scene."""
    name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
    role: str = Field(description="Main actions and/or motivations of the character in this scene")
    importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
    emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")

class SceneDetails(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters. Don't include any unnamed characters.")

def get_scenedetails_prompt(scene):
    scene_prompt = f'''List the key details in this scene: {scene}. Format your response like SceneDetails:
                    class CharacterInfo(BaseModel):
                        """Get information about a character in this scene."""
                        name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
                        role: str = Field(description="Main actions and/or motivations of the character in this scene")
                        importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
                        emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
                        sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
                        quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")

                    class SceneDetails(BaseModel):
                        """Get overall information about the scene."""
                        location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
                        importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
                        conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
                        characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters.  Don't include any unnamed characters.")'''
    return scene_prompt

scene_details_llm = llm.with_structured_output(SceneDetails)

In [14]:
def get_scene_info(chapters, chapter_num):
    cur_chapter = chapters[chapter_num]

    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            if model_type == "gemini":
                prompt = get_scenelist_prompt(cur_chapter)
                chapter_scenes = llm3.invoke(prompt,)
                chapter_scenes = format_gemini_response(chapter_scenes)
            else:
                chapter_scenes = scene_llm.invoke(cur_chapter)
            invalid_input = False
        except:
            invalid_input = True
            
    if model_type == "gemini":
        chapter_scenes_list = [DotDict(scene) for scene in chapter_scenes['scenelist']]
    else: 
        chapter_scenes_list = chapter_scenes.scenelist
    formatted_chapter_scenes = []

    for scene in chapter_scenes_list:
        scene_title = scene.scene
        scene_summary = scene.summary
        scene_first_line = scene.firstline
        scene_last_line = scene.lastline

        print(f"FIRST LINE: {scene_first_line}")
        print(f"LAST LINE: {scene_last_line}")
        print()

        formatted_scene = {
            "title": scene_title,
            "summary": scene_summary,
            "chapter": markers[chapter_num],
            "first_line": scene_first_line,
            "last_line": scene_last_line
        }
        formatted_chapter_scenes.append(formatted_scene)
        formatted_chapter = cur_chapter.replace("\n", " ")
        # replace weird quotes too
        formatted_chapter = formatted_chapter.replace("\"", " ").replace("'", " ")

    # split chapter into scenes based on formatted_chapter_scenes
    scenes = []
    cur_min_index = 0
    for i, scene in enumerate(formatted_chapter_scenes):
        cur_first_line = f"LINE {scene['first_line']}:"
        # cur_first_line = formatted_chapter_scenes[i]["first_line"].strip("...").replace("\"", " ").replace("'", " ")
        first_line_index = formatted_chapter.find(cur_first_line, cur_min_index)
        if first_line_index == -1:
            # see if first_line_index overlaps with last_line of previous scene
            other_possible_first_index = formatted_chapter.find(cur_first_line)
            if other_possible_first_index != -1:
                overlap = cur_chapter[other_possible_first_index:cur_min_index]
                print(f"SCENE {i}")
                print("\nPREVIOUS SCENE")
                print(scenes[i-1])
                print("\nOVERLAP")
                print(overlap)
                print()
            first_line_index = cur_min_index
        cur_last_line = f"LINE {scene['last_line'] + 1}:"
        # cur_last_line = formatted_chapter_scenes[i]["last_line"].strip("...").replace("\"", " ").replace("'", " ")
        last_line_index = formatted_chapter.find(cur_last_line, first_line_index)
        # last_line_index += len(cur_last_line)
        if last_line_index == -1:
            last_line_index = len(cur_chapter)
        scene_text = cur_chapter[first_line_index:last_line_index].strip()
        # remove line numbers
        scene_text = re.sub(r'LINE \d+: ', '', scene_text)
        scenes.append(scene_text)
        cur_min_index = last_line_index

    # add scene text to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        formatted_chapter_scenes[i]["text"] = scenes[i]

    return scenes, formatted_chapter_scenes

In [15]:
def get_scene_details(scenes, formatted_chapter_scenes):
    all_scene_details = []
    for i, scene in enumerate(scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        if len(scene_text) == 0:
            all_scene_details.append([])
            continue
        # get scene details for each scene
        cur_scene = scene
        invalid_input = True
        # if model_type == 'gemini':
        #     prompt = get_scenedetails_prompt(cur_scene)
        #     scene_details = llm.invoke(prompt)
        #     print(scene_details)
        #     scene_details = format_gemini_response(scene_details)
        #     print(scene_details)
        #     scene_details = DotDict(scene_details)
        # else:
        while invalid_input:
            try:
                scene_details = scene_details_llm.invoke(cur_scene)
                invalid_input = False
            except:
                invalid_input = True
        all_scene_details.append(scene_details)

    # add details to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        text_length = len(scene_text)
        num_lines = formatted_chapter_scenes[i]["last_line"] - formatted_chapter_scenes[i]["first_line"] + 1
        # first_line = formatted_chapter_scenes[i]["first_line"]
        # first_line_length = len(first_line)
        # last_line = formatted_chapter_scenes[i]["last_line"]
        # last_line_length = len(last_line)

        # if text_length == 0 or text_length < first_line_length + last_line_length:
        if text_length == 0:
            # no scene text or scene text is too short
            continue
        
        cur_scene_details = all_scene_details[i]
        formatted_chapter_scenes[i]["length"] = text_length
        formatted_chapter_scenes[i]["num_lines"] = num_lines
        formatted_chapter_scenes[i]["location"] = cur_scene_details.location
        formatted_chapter_scenes[i]["importance"] = cur_scene_details.importance
        formatted_chapter_scenes[i]["conflict"] = cur_scene_details.conflict
        formatted_character_info = []

        characters = cur_scene_details.characters
        # if model_type == "gemini":
        #     characters = [DotDict(character) for character in characters]
        for character in characters:
            character_info = {
                "name": character.name,
                "role": character.role,
                "importance": character.importance,
                "emotion": character.emotion,
                "sentiment": character.sentiment,
                "quote": character.quote
            }
            formatted_character_info.append(character_info)
        formatted_chapter_scenes[i]["characters"] = formatted_character_info

    return formatted_chapter_scenes

In [21]:
# get scene info for each chapter
for i, marker in enumerate(markers):
    if i != 8:
        continue
    print(f"Chapter {marker} start.")
    scenes, formatted_chapter_scenes = get_scene_info(chapters, i)
    print(f"Chapter {marker} scenes done.")
    formatted_chapter_scenes = get_scene_details(scenes, formatted_chapter_scenes)
    print(f"Chapter {marker} details done.")

    # remove scenes with no text
    formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if scene["text"] != "" and "characters" in scene]
    # remove characters with no quote
    for scene in formatted_chapter_scenes:
        scene["characters"] = [character for character in scene["characters"] if "(" not in character["quote"] != "" and "<" not in character["name"] and "unnamed" not in character["name"].lower()]
        # print(scene)
    
    # remove scenes with no characters
    formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if len(scene["characters"]) > 0]

    # save as json
    with open(f"json/{story_name}/{marker}.json", "w") as f:
        json.dump(formatted_chapter_scenes, f, indent=4)

    print(f"Chapter {marker} done.\n")

Chapter Chapter 9 start.
FIRST LINE: 1
LAST LINE: 20

FIRST LINE: 21
LAST LINE: 37

FIRST LINE: 38
LAST LINE: 57

FIRST LINE: 58
LAST LINE: 73

FIRST LINE: 74
LAST LINE: 107

FIRST LINE: 108
LAST LINE: 129

FIRST LINE: 130
LAST LINE: 187

Chapter Chapter 9 scenes done.
Chapter Chapter 9 details done.
Chapter Chapter 9 done.



In [14]:
# rank each scene by importance
# and within each scene, the characters

for i, marker in enumerate(markers):
    print(marker)
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    importances = []
    conflicts = []
    # extract importance from each scene
    for j, scene in enumerate(scenes):
        importances.append((j, scene["importance"]))
        conflicts.append((j, scene["conflict"]))
        # now extract character importances
        character_importances = []
        for k, character in enumerate(scene["characters"]):
            character_importances.append((k, character["importance"]))
        # sort character importances
        sorted_character_importances = sorted(character_importances, key=lambda x: x[1], reverse=True)
        # add importance_rank to each character
        for k, (l, _) in enumerate(sorted_character_importances):
            scenes[j]["characters"][l]["importance_rank"] = k+1
        # add number of each scene
        scenes[j]["number"] = j+1
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each scene
    for k, (j, _) in enumerate(sorted_importances):
        scenes[j]["importance_rank"] = k+1
    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each scene
    for k, (j, _) in enumerate(sorted_conflicts):
        scenes[j]["conflict_rank"] = k+1
    
    # print results
    # for j, scene in enumerate(scenes):
    #     print(scenes[j]["title"], scenes[j]["importance_rank"])
    #     print("--------------------------------")
    #     for k, character in enumerate(scene["characters"]):
    #         print(character["name"], character["importance_rank"])
    #     print()

    # save as json
    with open(f"json/{story_name}/{marker}.json", "w") as f:
        json.dump(scenes, f, indent=4)

Chapter 1
Chapter 2
Chapter 3
Chapter 4
Chapter 5
Chapter 6
Chapter 7
Chapter 8
Chapter 9
Chapter 10
Chapter 11
Chapter 12
Chapter 13
Chapter 14
Chapter 15
Chapter 16
Chapter 17
Chapter 18
Chapter 19
Chapter 20
Chapter 21
Chapter 22
Chapter 23
Chapter 24


In [15]:
# combine all chapters into one json file
all_scenes = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    all_scenes += scenes

with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(all_scenes, f, indent=4)

In [16]:
# collect all unique characters and locations for each chapter
all_characters = {}
all_locations = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    for scene in scenes:
        for character in scene["characters"]:
            char = character["name"]
            if char not in all_characters:
                all_characters[char] = character["role"]
        location = scene["location"]
        if location not in all_locations:
            all_locations.append(location)

In [17]:
len(all_characters), len(all_locations)

(81, 141)

In [18]:
all_characters

{'R. Walton': 'Expresses excitement and determination for his voyage',
 'Margaret': "Recipient of Walton's letter, represents familial concern",
 'Robert Walton': 'Narrator and explorer preparing for a voyage',
 'Mrs. Saville': 'Recipient of the letter and sister of Robert Walton',
 'R.W.': 'Writing a letter to his sister',
 'Captain Walton': 'Narrator and rescuer of the stranger',
 'The Stranger': 'Mysterious man rescued from ice',
 'Victor Frankenstein': 'Narrator and son of distinguished family',
 'Caroline Beaufort': "Victor's mother and a compassionate figure",
 'Alphonse Frankenstein': "Victor's father and a respected public figure",
 'Elizabeth Lavenza': "Victor's adopted sister and companion",
 'Beaufort': "Caroline's father and a friend of Victor's father",
 'Elizabeth': 'Calm and contemplative companion',
 'Narrator': 'Curious and passionate seeker of knowledge',
 'Henry Clerval': 'Best friend and adventurous spirit',
 'Clerval': 'Reflects on moral relations and heroism',
 "V

In [19]:
all_locations

['Petersburgh',
 'Archangel, Russia',
 'On a ship at sea',
 'Ship surrounded by ice',
 'Geneva',
 'Swiss home',
 'Geneva countryside',
 'Childhood home',
 "Clerval's reflections on life",
 'Childhood memories',
 'baths near Thonon',
 'Home of the narrator',
 'Geneva schools',
 'laboratory of alchemy',
 'house near Belrive',
 "young scholar's study",
 'University of Ingolstadt',
 'sickroom of Elizabeth',
 'Ingolstadt',
 "University professors' office",
 'University lecture room',
 "M. Waldman's study",
 "M. Waldman's laboratory",
 'Ingolstadt University',
 'anatomy study',
 'laboratory of Victor Frankenstein',
 'Unknown location',
 "Victor Frankenstein's laboratory",
 'a secluded laboratory',
 "Frankenstein's laboratory",
 'Ingolstadt streets',
 'Ingolstadt inn',
 "Victor's college apartment",
 "Victor's room",
 "Victor's recovery room",
 "Victor's sick room",
 "Victor's sickroom",
 'University professors meeting',
 'University of Natural Science',
 'Ingolstadt countryside',
 'Frankenst

In [20]:
# Assuming your JSON file is stored at 'all.json'
with open(f"json/{story_name}/all.json") as f:
    json_data = json.load(f)

prompt = """
You are given a list of characters with some duplicates or alternate names.
Your task is to group all the alternate names for each unique character under a single full name, using the descriptions as guidance.
Don't keep characters like "Narrator" unless they are never referred to by another name.
Separate characters that are related to others (e.g., "Bob" and "Ms. Doe (Bob's mom)" and "Bob's sister") into separate entries.

First create a new character list by removing all duplicates and alternate names and only keeping the full name for each character.

Then, output a JSON dictionary where the key is each character in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original character list and should only appear once in the output.

For example:
{{
    "Bob Smith": ["Bob", "Bobby", "Robert"],
    "Jane Doe": ["Jane", "Ms. Doe", "Bob's mother"]
}}
"""

location_prompt = """
You are given a list of locations with some duplicates or alternate names.
Your task is to group all the alternate or similiar names for each unique location under a single location name.

First create a new location list by removing all duplicates and alternate names and only keeping the full name for each location.
If one location is inside another or nearby to it, only keep the larger location (e.g., "Bob's bedroom", "Bob's garden", and "Bob Smith's castle" can be listed under "Bob's house").
Then, output a JSON dictionary where the key is each location in your new list and the value is a list of all possible alternate location names.
All locations should be taken directly from the original location list and should only appear once in the output dict.

For example:
{{
    "Los Angeles": ["LA", "City of Angels", "Cafe in Los Angeles"],
    "Bob's house": ["Bob's bedroom", "Bob Smith's garden", "Bob's castle"]
}}
"""

# if using llm2 (claude)
prompt += "Just output the JSON dictionary as the final result without any additional information."
location_prompt += "Just output the JSON dictionary as the final result without any additional information."

prompt_template = ChatPromptTemplate.from_messages([("system", prompt), ("human", "{input}")])
location_prompt_template = ChatPromptTemplate.from_messages([("system", location_prompt), ("human", "{input}")])

In [21]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
final_prompt = prompt_template.invoke(f"character list: {all_characters}")
# character_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
character_llm = llm2
# character_llm = llm
output = character_llm.invoke(final_prompt)


In [22]:
output

AIMessage(content='{\n    "Robert Walton": ["R. Walton", "R.W.", "Captain Walton"],\n    "Margaret Saville": ["Margaret", "Mrs. Saville"],\n    "Victor Frankenstein": ["Victor", "Narrator", "narrator", "Protagonist", "the protagonist", "M. Frankenstein"],\n    "The Creature": ["The Stranger", "Creature", "the creature", "the monster", "Frankenstein\'s Creature"],\n    "Caroline Beaufort": ["Mother"],\n    "Alphonse Frankenstein": ["Victor\'s Father", "Father Frankenstein", "Father", "Victor\'s father"],\n    "Elizabeth Lavenza": ["Elizabeth", "Elizabeth Frankenstein"],\n    "Beaufort": [],\n    "Henry Clerval": ["Clerval", "Henry"],\n    "M. Krempe": [],\n    "M. Waldman": [],\n    "William Frankenstein": ["Victor\'s brother"],\n    "Ernest Frankenstein": [],\n    "Justine Moritz": [],\n    "De Lacey": ["the old man", "Old Man", "Old man", "The Blind Father"],\n    "Agatha": ["Young Girl", "Young woman"],\n    "Felix": ["Young Man", "Young man"],\n    "Safie": ["Young Woman", "The Turk

In [23]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
final_location_prompt = location_prompt_template.invoke(f"location list: {all_locations}")
# location_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
location_llm = llm2
# location_llm = llm
location_output = location_llm.invoke(final_location_prompt)

In [24]:
location_output

AIMessage(content='{\n  "Petersburgh": [],\n  "Archangel, Russia": [],\n  "Ship at sea": ["On a ship at sea", "Ship surrounded by ice"],\n  "Geneva": ["Swiss home", "Geneva countryside", "Childhood home", "Home of the narrator", "Geneva schools", "Geneva outskirts", "Frankenstein family home", "Victor\'s father\'s house", "Victor\'s family home", "family home", "Frankenstein\'s home", "Victor\'s home in Geneva", "Father\'s house", "Victor\'s home at night", "Geneva, May 18th", "Lake Geneva"],\n  "Thonon": ["baths near Thonon"],\n  "Belrive": ["house near Belrive", "Belrive house by the lake"],\n  "University of Ingolstadt": ["laboratory of alchemy", "young scholar\'s study", "Ingolstadt", "University professors\' office", "University lecture room", "M. Waldman\'s study", "M. Waldman\'s laboratory", "Ingolstadt University", "anatomy study", "laboratory of Victor Frankenstein", "Victor Frankenstein\'s laboratory", "a secluded laboratory", "Frankenstein\'s laboratory", "Ingolstadt streets

In [25]:
# if model_type == "claude":
output_formatted = output.content
# turn into json
output = json.loads(output_formatted)
location_output_formatted = location_output.content
# turn into json
location_output = json.loads(location_output_formatted)

In [26]:
# Function to find the correct key based on a search string, ensuring whole-word matches
def find_key_from_alias(text, character_dict):
    for key, aliases in character_dict.items():
        for alias in sorted(aliases, key=len, reverse=True):
            # Match the alias as a whole word, and ensure no partial matches within other words
            pattern = rf'\b{re.escape(alias)}\b(?!\'s)'  # (?!\'s) ensures no match for possessives like Mitsuha's
            if re.search(pattern, text):
                return key  # Return the key if the alias is found as a whole word
    return text  # Return the original text if no match is found

# Load the scenes
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}
capitalized_location_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in location_output.items()}

# Process each scene
for scene in scenes:
    # Process characters
    for character in scene["characters"]:
        char = string.capwords(character["name"])
        found_char = False

        # Check if the character's name is a key in the dictionary
        if char in capitalized_output:
            found_char = True
            character["name"] = char  # Capitalize the character name
        
        # If not found in the dictionary, check the aliases
        if not found_char:
            # Find the right key for the alias (if exists)
            matched_key = find_key_from_alias(char, capitalized_output)
            if matched_key != char:
                character["name"] = matched_key
                found_char = True
        if not found_char:
            character["name"] = char

    # Process location
    loc = string.capwords(scene["location"])
    found_loc = False

    # Check if the location is a key in the location dictionary
    if loc in capitalized_location_output:
        found_loc = True
        scene["location"] = loc  # Capitalize the location

    # If not found in the dictionary, check the aliases
    if not found_loc:
        matched_key = find_key_from_alias(loc, capitalized_location_output)
        if matched_key != loc:
            scene["location"] = matched_key
            found_loc = True
    if not found_loc:
        scene["location"] = loc

# Save the updated scenes
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

In [27]:
# now list all characters and locations again
# collect all unique characters and locations for each chapter
all_characters_new = {}
all_locations_new = []
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
for scene in scenes:
    for character in scene["characters"]:
        char = character["name"]
        if char not in all_characters_new:
            all_characters_new[char] = 1
        else:
            all_characters_new[char] += 1
    location = scene["location"]
    if location not in all_locations_new:
        all_locations_new.append(location)

In [28]:
len(all_characters_new), len(all_locations_new)

(40, 58)

In [29]:
all_characters_new 

{'Robert Walton': 3,
 'Margaret Saville': 3,
 'R.w.': 1,
 'The Creature': 32,
 'Victor Frankenstein': 126,
 'Caroline Beaufort': 2,
 'Alphonse Frankenstein': 20,
 'Elizabeth Lavenza': 23,
 'Beaufort': 1,
 'Henry Clerval': 21,
 'Man Of Great Research': 1,
 'M. Krempe': 3,
 'M. Waldman': 5,
 'William Frankenstein': 4,
 'Ernest Frankenstein': 2,
 'Justine Moritz': 4,
 'De Lacey': 8,
 'Agatha': 9,
 'Felix': 17,
 'Youth': 1,
 'Stranger': 1,
 'Safie': 8,
 'The Turk': 4,
 "Safie's Mother": 1,
 'Confidential Servant': 1,
 'Werter': 1,
 'Plutarch': 1,
 'Adam': 1,
 'Satan': 1,
 'Child': 1,
 'The Woman': 1,
 'Hoarse-voiced Man': 1,
 'Ill-looking Man': 1,
 'Mr. Kirwin': 6,
 'Daniel Nugent': 2,
 'Old Woman': 1,
 'The Companions': 1,
 'Criminal Judge': 1,
 'Villagers': 1,
 'Walton': 1}

In [30]:
all_locations_new

['Petersburgh',
 'Archangel, Russia',
 'Ship At Sea',
 'Geneva',
 "Clerval's Reflections On Life",
 'Childhood Memories',
 'Thonon',
 'University Of Ingolstadt',
 'Belrive',
 'Sickroom Of Elizabeth',
 'Unknown Location',
 "Frankenstein's Room",
 'Journey To Native Town',
 'A Dimly Lit Room',
 'Courtroom',
 'Prison',
 "Victor's Tortured Mind",
 "Frankenstein's Tortured Mind",
 'House Of Mourning',
 "Victor's Inner Turmoil",
 'Alpine Valleys',
 'Forest',
 'Cottage In The Woods',
 'Transparent Pool',
 'Garden In Spring',
 'Unknown Forest Clearing',
 'Springtime Landscape',
 'Desert Turning Into Bloom',
 "Merchant's Home",
 "Safie's Background Story",
 'Paris',
 "Turk's Home",
 'Cottage In Germany',
 'Leghorn, Italy',
 'Damp Grass',
 'Near The Cottage',
 'Jura Mountains',
 'A Secluded Barn',
 'A Dark, Secluded Place',
 'Wilds Of The Field',
 'Mountain Ice Caves',
 'Chamounix',
 'Journey To England',
 'Scotland',
 'Moonlit Room By The Sea',
 'Solitary Beach At Night',
 'Irish Town',
 "Magis

In [31]:
# Pydantic
class ChapterSummary(BaseModel):
    """Summarize a chapter based on scenes."""
    summary: str = Field(description="A brief, 1-line summary of the chapter")

chapter_llm = llm.with_structured_output(ChapterSummary)

In [32]:
# remove any markers without any scenes
markers_to_remove = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    if len(scenes) == 0:
        markers_to_remove.append(marker)

for marker in markers_to_remove:
    markers.remove(marker)

In [33]:
# create summary json of each chapter
chapter_summaries = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
        
    with open(f"chapters/{story_name}/{marker}.txt") as f:
        chapter = f.read()

    # get summary of chapter
    scene_summaries = " ".join([scene["summary"] for scene in scenes])
    summary = chapter_llm.invoke(scene_summaries)

    chapter_summary = {
        "chapter": marker,
        "scenes": len(scenes),
        "length": len(chapter),
        "num_lines": len(chapter.split("\n")),
        "summary": summary.summary,
        "importance": [],
        "conflict": [],
        "locations": {},
        "characters": {}
    }
    chapter_summaries.append(chapter_summary)
    
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
    for scene in scenes:
        chapter = scene["chapter"]
        chapter_summary = chapter_summaries[markers.index(chapter)]
        # add location to chapter summary
        location = scene["location"]
        chapter_summary_locations = chapter_summary["locations"]
        if location not in chapter_summary_locations:
            chapter_summary_locations[location] = 1
        else:
            chapter_summary_locations[location] += 1
        # add importance to chapter summary
        importance = scene["importance"]
        chapter_summary["importance"].append(importance)
        # add conflict to chapter summary
        conflict = scene["conflict"]
        chapter_summary["conflict"].append(conflict)
        # add characters to chapter summary
        characters = scene["characters"]
        for character in characters:
            char = character["name"]
            chapter_summary_characters = chapter_summary["characters"]
            if char not in chapter_summary_characters:
                chapter_summary_characters[char] = 1
            else:
                chapter_summary_characters[char] += 1

# now average importance and conflict for each chapter
for chapter_summary in chapter_summaries:
    if len(chapter_summary["importance"]) == 0:
       print(f"Chapter {chapter_summary['chapter']} has no scenes.")
    chapter_summary["importance"] = round(sum(chapter_summary["importance"]) / len(chapter_summary["importance"]), 2)
    chapter_summary["conflict"] = round(sum(chapter_summary["conflict"]) / len(chapter_summary["conflict"]), 2)

# rank each chapter by importance and conflict
importances = []
conflicts = []
for chapter_summary in chapter_summaries:
    importances.append((chapter_summary["chapter"], chapter_summary["importance"]))
    conflicts.append((chapter_summary["chapter"], chapter_summary["conflict"]))
    
# sort importances
sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
# add importance_rank to each chapter
for k, (j, _) in enumerate(sorted_importances):
    chapter_summaries[markers.index(j)]["importance_rank"] = k+1

# sort conflicts
sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
# add conflict_rank to each chapter
for k, (j, _) in enumerate(sorted_conflicts):
    chapter_summaries[markers.index(j)]["conflict_rank"] = k+1

In [34]:
chapter_summaries

[{'chapter': 'Chapter 1',
  'scenes': 5,
  'length': 47260,
  'num_lines': 628,
  'summary': 'Robert Walton writes letters to his sister Margaret detailing his expedition, longing for friendship, and the mysterious encounter with Victor Frankenstein, who begins to share his family history.',
  'importance': 0.84,
  'conflict': 0.44,
  'locations': {'Petersburgh': 1,
   'Archangel, Russia': 1,
   'Ship At Sea': 2,
   'Geneva': 1},
  'characters': {'Robert Walton': 3,
   'Margaret Saville': 3,
   'R.w.': 1,
   'The Creature': 1,
   'Victor Frankenstein': 1,
   'Caroline Beaufort': 1,
   'Alphonse Frankenstein': 1,
   'Elizabeth Lavenza': 1,
   'Beaufort': 1},
  'importance_rank': 15,
  'conflict_rank': 20},
 {'chapter': 'Chapter 2',
  'scenes': 11,
  'length': 14461,
  'num_lines': 189,
  'summary': "Victor Frankenstein's childhood is marked by his contrasting relationship with Elizabeth and Henry Clerval, leading him from a joyful upbringing to a dangerous obsession with knowledge, part

In [35]:
all_json = {}
all_json["title"] = story_info["title"]
all_json["type"] = story_info["type"]
if (all_json["type"] == "Movie"):
    all_json["director"] = story_info["director"]
else:
    all_json["author"] = story_info["author"]
all_json["year"] = story_info["year"]
all_json["url"] = story_info["url"]
all_json["image"] = story_info["image"]
all_json["num_chapters"] = len(chapters)
all_json["num_scenes"] = len(scenes)
all_json["num_characters"] = len(all_characters_new)
all_json["num_locations"] = len(all_locations_new)
all_json["chapters"] = chapter_summaries
all_json["scenes"] = scenes

In [36]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in the story."""
    name: str = Field(description="Name of character.")
    quote: str = Field(description="Direct quote from the story that the character says or that describes them.")
    group: str = Field(description="Group that this character belongs to (e.g., main characters, upper class, talking animals). There should be at least 2 different groups of characters.")
    # color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    # explanation: str = Field(description="Explanation why this color was chosen for this character.")

class CharacterList(BaseModel):
    """Get overall information about the story."""
    # characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")
    characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added.")

class ColorInfo(BaseModel):
    """Pick a color + explain why for each character"""
    name: str = Field(description="Name of character.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this character.")

class ColorList(BaseModel):
    """List of color info for characters"""
    colorlist: list[ColorInfo] = Field(description="List of color details for each character. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")

characterlist_llm = llm.with_structured_output(CharacterList)
colorlist_llm = llm.with_structured_output(ColorList)

In [37]:
# sort all_characters_new by count, descending
sorted_characters = dict(sorted(all_characters_new.items(), key=lambda item: item[1], reverse=True))
# take top 20 characters
top_characters = dict(list(sorted_characters.items())[:20])
top_characters

{'Victor Frankenstein': 126,
 'The Creature': 32,
 'Elizabeth Lavenza': 23,
 'Henry Clerval': 21,
 'Alphonse Frankenstein': 20,
 'Felix': 17,
 'Agatha': 9,
 'De Lacey': 8,
 'Safie': 8,
 'Mr. Kirwin': 6,
 'M. Waldman': 5,
 'William Frankenstein': 4,
 'Justine Moritz': 4,
 'The Turk': 4,
 'Robert Walton': 3,
 'Margaret Saville': 3,
 'M. Krempe': 3,
 'Caroline Beaufort': 2,
 'Ernest Frankenstein': 2,
 'Daniel Nugent': 2}

In [38]:
character_arr = []
cur_arr = []
max_chars = 20

char_names = list(all_characters_new.keys())
# char_names = list(top_characters.keys())
for i, c in enumerate(char_names):
    cur_arr.append(c)
    if len(cur_arr) == max_chars or i == len(char_names) - 1:
        character_arr.append(cur_arr)
        cur_arr = []

for arr in character_arr:
    print(len(arr))


20
20


In [39]:
# read in scenes from all_json
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

In [40]:
all_res = []
used_colors = []
for i, arr in enumerate(character_arr):
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        # filter all_json to only include characters in arr
        new_scenes = []
        # for scene in all_json["scenes"]:
        for scene in scenes:
            include = False
            for character in scene["characters"]:
                if character["name"] in arr:
                    include = True
                    break
            if include:
                new_scenes.append(scene)

        # take middle 126000 characters of new_scenes
        new_scenes_str = json.dumps(new_scenes)
        new_scenes_str = new_scenes_str[:126000]

        try:
            # prompt = f"characters: {arr}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(arr)} characters as in the original list. Make sure each character is assigned a unique color."
            prompt = f"characters: {arr}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(arr)} characters as in the original list."
            # if len(used_colors) > 0:
            #     prompt += f"\ndon't use any of these colors: {used_colors}"
            res = characterlist_llm.invoke(prompt)
            all_res.append(res)
            # int_list = res.characterlist
            # for j, char in enumerate(int_list):
            #     used_colors.append(char.color)
            print(f"Chunk {i} done.")
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True

Chunk 0 done.
Chunk 1 done.


In [41]:
# combine characterlists into one
characters = []
for res in all_res:
    print(len(res.characterlist))
    characters += res.characterlist

len(characters)

20
20


40

In [42]:
character_names = [char.name for char in characters]
for name in character_names:
    if name not in char_names:
        print("not in orig list:", name)

need_to_add = []
for name in char_names:
    if name not in character_names:
        print("not in llm output:", name)
        need_to_add.append(name)

# find duplicate names
unique_names = []
duplicate_indices = []
for i, name in enumerate(character_names):
    if name not in unique_names:
        unique_names.append(name)
    else:
        print("duplicate:", name, i)
        duplicate_indices.append(i)


In [43]:
# now add colors for top characters
top_char_names = list(top_characters.keys())

all_res_colors = []
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    # filter all_json to only include characters in arr
    new_scenes = []
    for scene in scenes:
        include = False
        for character in scene["characters"]:
            if character["name"] in top_char_names:
                include = True
                break
        if include:
            new_scenes.append(scene)

    # take middle 126000 characters of new_scenes
    new_scenes_str = json.dumps(new_scenes)
    new_scenes_str = new_scenes_str[:126000]

    try:
        prompt = f"characters: {top_char_names}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(top_char_names)} characters as in the original list. Make sure each character is assigned a unique color."
        res = colorlist_llm.invoke(prompt)
        all_res_colors.append(res)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [44]:
# find duplicate colors
colorlist = []
for res in all_res_colors:
    colorlist += res.colorlist
    
unique_colors = []
duplicate_colors = []
for i, char in enumerate(colorlist):
    color = char.color
    if color not in unique_colors:
        unique_colors.append(color)
    else:
        print("duplicate color:", color, i)
        duplicate_colors.append(i)

duplicate color: rgb(255, 99, 71) 12


In [45]:
colorinfo = {}
for c in colorlist:
    colorinfo[c.name] = {
        "color": c.color,
        "explanation": c.explanation
    }

colorinfo

{'Victor Frankenstein': {'color': 'rgb(255, 99, 71)',
  'explanation': "This vibrant red-orange color represents Victor's intense passion and ambition, as well as the horror and regret he feels after creating the creature."},
 'The Creature': {'color': 'rgb(70, 130, 180)',
  'explanation': "This steel blue color symbolizes the Creature's melancholy and despair, reflecting his tragic existence and longing for acceptance."},
 'Elizabeth Lavenza': {'color': 'rgb(255, 228, 196)',
  'explanation': "This soft peach color embodies Elizabeth's gentle and nurturing nature, representing her warmth and compassion."},
 'Henry Clerval': {'color': 'rgb(144, 238, 144)',
  'explanation': "This light green color signifies Henry's adventurous spirit and optimism, contrasting with Victor's darker obsessions."},
 'Alphonse Frankenstein': {'color': 'rgb(255, 215, 0)',
  'explanation': "This golden yellow color reflects Alphonse's integrity and warmth as a father, symbolizing his supportive and loving natur

In [46]:
# remove duplicates
characters = [char for i, char in enumerate(characters) if i not in duplicate_indices]
print(len(characters))

character_info = []
for c in characters:

    # see if character is in color_info
    name = c.name
    color = ""
    explanation = ""
    if name in colorinfo:
        color = colorinfo[name]["color"]
        explanation = colorinfo[name]["explanation"]
    
    character_info.append({
        "name": name,
        "quote": c.quote,
        "group": c.group,
        "color": color,
        "explanation": explanation
    })

character_info

40


[{'name': 'Robert Walton',
  'quote': 'I feel my heart glow with an enthusiasm which elevates me to heaven.',
  'group': 'main characters',
  'color': 'rgb(0, 255, 127)',
  'explanation': "This spring green color symbolizes Walton's adventurous spirit and hopefulness, representing his quest for discovery."},
 {'name': 'Margaret Saville',
  'quote': 'Heaven shower down blessings on you.',
  'group': 'main characters',
  'color': 'rgb(255, 20, 147)',
  'explanation': "This deep pink color embodies Margaret's concern and support for her brother, representing familial love."},
 {'name': 'R.w.',
  'quote': 'But success _shall_ crown my endeavours.',
  'group': 'main characters',
  'color': '',
  'explanation': ''},
 {'name': 'The Creature',
  'quote': 'Unhappy man! Do you share my madness?',
  'group': 'main characters',
  'color': 'rgb(70, 130, 180)',
  'explanation': "This steel blue color symbolizes the Creature's melancholy and despair, reflecting his tragic existence and longing for ac

In [35]:
# in final data, for top characters, replace existing color + explanation with new ones

# read in final_data.json
# with open(f"json/{story_name}/final_data.json") as f:
#     final_data = json.load(f)

# character_names = [char["name"] for char in character_info]

# for i, c in enumerate(final_data["characters"]):
#     # see if matching character in character_info
#     if c["name"] in character_names:
#         char_index = character_names.index(c["name"])
#         final_data["characters"][i]["color"] = character_info[char_index]["color"]
#         final_data["characters"][i]["explanation"] = character_info[char_index]["explanation"]
#     else:
#         # clear color and explanation
#         final_data["characters"][i]["color"] = ""
#         final_data["characters"][i]["explanation"] = ""

# # save final_data.json
# with open(f"json/{story_name}/final_data.json", "w") as f:
#     json.dump(final_data, f, indent=4)

In [47]:
# Pydantic
class LocationInfo(BaseModel):
    """Get information about a location in the story."""
    name: str = Field(description="Name of location.")
    quote: str = Field(description="Direct quote from the story that describes this location")
    emoji: str = Field(description="Emoji that represents this location (e.g., 🏰). Make sure each location has a different emoji.")

class LocationList(BaseModel):
    """Get overall information about the story."""
    locationList: list[LocationInfo] = Field(description="List of location details. One entry per location.")

locationList_llm = llm.with_structured_output(LocationList)

In [53]:
# read in scenes from all_json
# story_name = "yourname"
# with open(f"json/{story_name}/final_data.json") as f:
#     all_json = json.load(f)

In [54]:
# locations = all_json["locations"]
# all_locations_new = [loc["name"] for loc in locations]
# character_info = all_json["characters"]

In [55]:
cropped_json = json.dumps(all_json)[:126000]
location_prompt = f"locations: {all_locations_new}\nstory info: {cropped_json}\nyour output should contain exactly the same {len(all_locations_new)} locations as in the original list."
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    try:
        res = locationList_llm.invoke(location_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [56]:
location_info = []
for l in res.locationList:
    location_info.append({
        "name": l.name,
        "quote": l.quote,
        "emoji": l.emoji
    })

print(len(location_info))
print(location_info)

17
[{'name': 'Train', 'quote': "I don't recognize that ringtone, I think drowsily.", 'emoji': '🚆'}, {'name': 'Unknown Bedroom', 'quote': "I'm...a girl?", 'emoji': '🛏️'}, {'name': 'Itomori', 'quote': 'The deep-green mountains form their ranks under white clouds in a blue sky.', 'emoji': '🏞️'}, {'name': 'School', 'quote': "You seriously don't remember?", 'emoji': '🏫'}, {'name': 'Miyamizu Shrine', 'quote': "Our braided cords hold a thousand years of Itomori's history.", 'emoji': '⛩️'}, {'name': 'Strange Bathroom', 'quote': 'What a weird dream.', 'emoji': '🚽'}, {'name': 'Tokyo', 'quote': 'This world is far too dazzling.', 'emoji': '🏙️'}, {'name': 'Restaurants', 'quote': 'So listen, after class today, wanna go to that cafe again?', 'emoji': '🍽️'}, {'name': 'Dressing Room', 'quote': 'This is amazing! Taki, wow!', 'emoji': '👗'}, {'name': "Taki's Workplace", 'quote': '...What is this?', 'emoji': '🏢'}, {'name': "Mitsuha's Home", 'quote': 'What the heck is this life??', 'emoji': '🏠'}, {'name': '

In [57]:
# add character and location info to all_json
all_json["characters"] = character_info
all_json["locations"] = location_info
all_json

{'title': 'Your Name',
 'type': 'Movie',
 'director': 'Makoto Shinkai',
 'year': '2016',
 'url': 'https://archive.org/stream/manga_Kimi_no_Na_wa/Kimi%20no%20Na%20wa%20%5BYen%20Press%5D_djvu.txt',
 'image': 'https://m.media-amazon.com/images/M/MV5BODRmZDVmNzUtZDA4ZC00NjhkLWI2M2UtN2M0ZDIzNDcxYThjL2ltYWdlXkEyXkFqcGdeQXVyNTk0MzMzODA@._V1_FMjpg_UX1000_.jpg',
 'num_chapters': 8,
 'num_scenes': 55,
 'num_characters': 30,
 'num_locations': 17,
 'chapters': [{'chapter': 'Chapter Two The Beginning',
   'scenes': 10,
   'length': 39116,
   'num_lines': 583,
   'summary': 'The protagonist wakes up in a different body, navigates her new life with her family and friends, faces embarrassing moments at school, and participates in a traditional harvest festival, ultimately feeling disillusioned with her small town.',
   'importance': 0.78,
   'conflict': 0.58,
   'locations': {'Train': 1,
    'Unknown Bedroom': 1,
    'Itomori': 4,
    'School': 2,
    'Miyamizu Shrine': 2},
   'characters': {'Taki Tac

In [58]:
# save as json
with open(f"json/{story_name}/final_data.json", "w") as f:
    json.dump(all_json, f, indent=4)